# Eval code for action-based VAE

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
# enable access to action-from-pose model
si_path = '/home/sam/repos/structuredinference/'
if si_path not in sys.path:
    sys.path.append(si_path)

In [ ]:
import json
from os import path

import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

from common import plot_confusion_matrix
from generate_seq_2d_ac_vae import load_data, parser
from common_pp.act_class_model import make_model
from common_pp.act_pre_common import one_hot_cat, classifier_transform, balance_aclass_ds, merge_actions

In [ ]:
with open('seq-2d-ac-vae/config.json', 'r') as fp:
    config = json.load(fp)
    # might actually be easier to just parse the arguments if I need anything else
    data_file = None
    args = parser.parse_args(config['args'])
    data_file = args.data_file
    seq_length = args.seq_length
    seq_skip = args.seq_skip

In [ ]:
def print_load(path):
    print('Loading', path)
    return load_model(path)
act_encoder = print_load(config['act_encoder_path'])
pose_encoder = print_load(config['pose_encoder_path'])
decoder = print_load(config['decoder_path'])

In [ ]:
_, _, _, _, db = load_data(data_file, seq_length, seq_skip)

## Expt 1: Classifying generated poses

See whether correct actions can be recovered from samples taken from the model (by training separate predictor to look at action-conditioned poses). This is a "minimum standard" for success (and one which the DMM does not meet).

In [ ]:
train_aclass_ds = db.get_aclass_ds(train=True)
val_aclass_ds = db.get_aclass_ds(train=False)
merge_map = {
    'attach leg 1': '*tach',
    'attach leg 2': '*tach',
    'attach leg 3': '*tach',
    'attach leg 4': '*tach',
    'detach leg 1': '*tach',
    'detach leg 2': '*tach',
    'detach leg 3': '*tach',
    'detach leg 4': '*tach',
    'spin in': 'spin',
    'spin out': 'spin',
    'n/a': None
}
# note that we're using unbalanced datasets
_, train_aclass_ds \
    = merge_actions(train_aclass_ds, merge_map, db.action_names)
aclass_target_names, val_aclass_ds \
    = merge_actions(val_aclass_ds, merge_map, db.action_names)

In [ ]:
def merge_act_inflow(merged_idx):
    """How many original actions map to the given merged
    action?"""
    merged_name = aclass_target_names[merged_idx]
    return len({
            k for k, v in merge_map.items() if v == merged_name
        })

def to_merged_idx(orig_idx):
    """What is the index of the merged action which the given
    original action points to."""
    orig_name = db.action_names[orig_idx]
    new_name = merge_map.get(orig_name, orig_name)
    if new_name is None:
        return None
    return aclass_target_names.index(new_name)

In [ ]:
def make_samples(samples_per_act):
    gen_Z = []
    gen_Y = []
    noise_dim = decoder.input_shape[-1]
    for act_idx in range(db.num_actions):
        merged_idx = to_merged_idx(act_idx)
        if merged_idx is None:
            # skip this action because it was removed
            continue
        inflow = merge_act_inflow(merged_idx)
        to_generate = int(np.ceil(samples_per_act / float(inflow)))
        mu, cov = act_encoder.predict(np.asarray([[act_idx] * seq_length]))
        nzs = np.random.randn(to_generate, noise_dim)
        zs = nzs * cov + mu
        gen_Z.extend(zs.tolist())
        gen_Y.extend([merged_idx] * to_generate)

    gen_Z = np.asarray(gen_Z)
    gen_Y = one_hot_cat(np.asarray(gen_Y), len(aclass_target_names))
    assert np.all(np.isfinite(gen_Z))
    assert np.all(np.isfinite(gen_Y))

    gen_X = classifier_transform(decoder.predict(gen_Z, batch_size=1024))
    assert np.all(np.isfinite(gen_X))

    return gen_X, gen_Y

In [ ]:
gen_X, gen_Y = make_samples(8192)
val_X, val_Y = make_samples(1024)

In [ ]:
print('gen_X:', gen_X.shape, 'gen_Y:', gen_Y.shape,
      'val_X:',  val_X.shape, 'val_Y:', val_Y.shape)

In [ ]:
# now try to fit a model to the recovered poses
model = make_model(gen_X.shape[1], gen_X.shape[2], len(aclass_target_names))
model.fit(gen_X,
          gen_Y,
          batch_size=64,
          nb_epoch=2,
          validation_data=(val_X, val_Y))

In [ ]:
Y_pred = model.predict(val_X, batch_size=10000).argmax(axis=-1)    
print('Results for sampled, action-conditioned poses:')
print(classification_report(val_Y.argmax(axis=-1), Y_pred, target_names=aclass_target_names))
cm = confusion_matrix(val_Y.argmax(axis=-1), Y_pred)
plot_confusion_matrix(cm, aclass_target_names)
plt.show()

Now let's do the same thing with the "real" action classifier.

In [ ]:
old_action_model_path = path.join(si_path, 'expt-ikeadb/chkpt-aclass/action-classifier-50-0.75.hdf5')
old_action_model = load_model(old_action_model_path)
sample_class_actions = old_action_model.output_shape[1]
sample_class_indim = old_action_model.input_shape[-1]
action_model = make_model(db.seq_length - 1,
                          sample_class_indim,
                          sample_class_actions)
action_model.set_weights(old_action_model.get_weights())

In [ ]:
Y_pred = action_model.predict(val_X, batch_size=10000).argmax(axis=-1)    
print('Results for sampled, action-conditioned poses:')
print(classification_report(val_Y.argmax(axis=-1), Y_pred, target_names=aclass_target_names))
cm = confusion_matrix(val_Y.argmax(axis=-1), Y_pred)
plot_confusion_matrix(cm, aclass_target_names)
plt.show()

## Expt 2: realism on action classifier dataset

Check whether pose sequences passed through the encoder are as realistic (in terms of recovered actions) as the corresponding originals. Will have to use action classification dataset for this.

Note that the input here is poses K, K-1 through 1, and the output is poses 1 through K. No actions are used at any point.

## Expt 3: overlaying poses

Try to do some completions. Layer them on top of the original videos so that I can see whether my model is actually working.